In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
pd.__version__

'0.25.1'

In [3]:
# Load split data frames

user = pd.read_csv('/home/burtenshaw/data/teen/beta/whatsapp/_6/wa_user_df.csv', index_col=0)
line = pd.read_csv('/home/burtenshaw/data/teen/beta/whatsapp/_6/wa_line_df.csv', index_col=0)
conv = pd.read_csv('/home/burtenshaw/data/teen/beta/whatsapp/_6/wa_conv_df.csv', index_col=0)

a_user = pd.read_csv('/home/burtenshaw/data/teen/alpha/redo/alpha_users.csv', index_col=0)
a_line = pd.read_csv('/home/burtenshaw/data/teen/alpha/redo/alpha_lines.csv', index_col=0)

# Encorporate validated matches
key = pd.read_csv('/home/burtenshaw/data/teen/beta/combined/combined_matches.csv', index_col=0)

,beta_contact_name,matched_chatter_id,match_intersection,AS_ALPHA_chatter_id,proposed_chatter_ids
wa_u_0,31 6 53901029,2321,0.0,[None],['2321']
wa_u_1,+32 473 94 69 36,2109,0.0,[None],['2109']
wa_u_2,+32 486 70 96 71,2108,0.0,[None],['2108']
wa_u_3,+32 489 09 32 72,2110,0.0,[None],['2110']
wa_u_4,31 6 53901029,2321,0.0,[None],['2321']
...,...,...,...,...,...
fb_u_2076,Zsofi Segers,NaN,NaN,NaN,NaN
fb_u_2077,Zuwena De Moor,118,0.0,['118'],['118']
fb_u_2078,Јован Лос Меренгуес,NaN,NaN,NaN,NaN
fb_u_2079,Мalik Abdul,NaN,NaN,NaN,NaN


In [25]:
# fields from conv
_c = conv[['line_idxs','users']]
_c['line_idxs'] = _c.line_idxs.apply(literal_eval)
_c = _c.explode('line_idxs')
_c['conv_idx'] = _c.index
_c.index = _c.line_idxs

/home/burtenshaw/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
# fields from user
# LOAD ALPHA DATA - Where Possible\
_u = user[['line_idxs','AS_ALPHA_chatter_id']]
_u['chatter_id'] = key.matched_chatter_id

/home/burtenshaw/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [27]:
_u = pd.merge(left=_u,right=a_user,left_on='chatter_id', right_index=True, how='left')

In [28]:
_u['line_idxs'] = _u.line_idxs.apply(literal_eval)
_u = _u.explode('line_idxs')
_u['user_idx'] = _u.index
_u.index = _u.line_idxs

In [29]:
_s = pd.DataFrame(list(line.source.apply(lambda x: x.split('/')[-1].split('_')[:4])),columns=['year','medium','conv_sex','conv_size'])

In [30]:
all_on_line = pd.concat([_u,_c,_s], axis=1)
all_on_line = pd.merge(left= all_on_line, right=line, left_index=True, right_index=True)

In [31]:
all_on_line.head()

,line_idxs,AS_ALPHA_chatter_id,chatter_id,gender,age,region,education,language,prof,prof_cat,...,raw_date,text,user,utc,source,line_idx.1,conv_idx_y,school_y,submitter,clean_submitter
wa_l_0,wa_l_0,['website 537'],website 537,V,14.0,Antwerpen,BSO,nederlands/vlaams,onthaalmama/stellingbouwer,6,...,"04-05-16, 21:03",Ik heb watsaaaap,kaylee elvidge,1459882980,data_WA/kta_denbiezerd_niel/2016_WhatsApp_same...,wa_l_0,wa_c_0,kta_denbiezerd_niel,kayleeelvidge2,kayleeelvidge
wa_l_1,wa_l_1,['965'],965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"04-05-16, 21:06",Kei goe,Mayaaa,1459883160,data_WA/kta_denbiezerd_niel/2016_WhatsApp_same...,wa_l_1,wa_c_0,kta_denbiezerd_niel,kayleeelvidge2,kayleeelvidge
wa_l_10,wa_l_10,['965'],965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"04-05-16, 21:16",Mijn mama hefd die juist gestiurt❤,Mayaaa,1459883760,data_WA/kta_denbiezerd_niel/2016_WhatsApp_same...,wa_l_10,wa_c_0,kta_denbiezerd_niel,kayleeelvidge2,kayleeelvidge
wa_l_100,wa_l_100,['965'],965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"08-05-16, 22:25","Oke,danku da gy der voor mij bent wjw💕",Mayaaa,1470428700,data_WA/kta_denbiezerd_niel/2016_WhatsApp_same...,wa_l_100,wa_c_0,kta_denbiezerd_niel,kayleeelvidge2,kayleeelvidge
wa_l_1000,wa_l_1000,['1351'],1351,V,15.0,Antwerpen,ASO,nederlands,in gesprek,in gesprek,...,"10-01-16, 21:22",Sorry moest da ff sture 😂❤,Sarie Casselo,1475349720,data_WA/sint_ursula_lier/2016_WhatsApp_same_pr...,wa_l_1000,wa_c_100,sint_ursula_lier,lorenooyens5,lorenooyens


In [32]:
all_on_line.columns

Index(['line_idxs', 'AS_ALPHA_chatter_id', 'chatter_id', 'gender', 'age',
       'region', 'education', 'language', 'prof', 'prof_cat', 'permission',
       'permission_parents', 'school_x', 'in_gesprek', 'lines_n', 'user_idx',
       'line_idxs', 'users', 'conv_idx_x', 'year', 'medium', 'conv_sex',
       'conv_size', 'Unnamed: 0', 'conv_n', 'line_n', 'raw_date', 'text',
       'user', 'utc', 'source', 'line_idx.1', 'conv_idx_y', 'school_y',
       'submitter', 'clean_submitter'],
      dtype='object')

In [33]:
all_on_line.drop(columns=['line_idxs','line_n'], inplace=True)

In [21]:
all_on_line.to_csv('/home/burtenshaw/data/teen/beta/whatsapp/_6/wa_all_on_line.csv')